In [35]:
#Importar o primeiro banco de dados de leitos
import pandas as pd
leitos_23 = pd.read_csv("Leitos_2023.csv", encoding="latin-1")
leitos_23 = leitos_23.drop(columns=["MOTIVO_DESABILITACAO","CNES","CO_TIPO_UNIDADE","DS_TIPO_UNIDADE","NATUREZA_JURIDICA","TP_GESTAO","RAZAO_SOCIAL","NO_COMPLEMENTO"])
display(leitos_23.head())









,COMP,REGIAO,UF,MUNICIPIO,NOME_ESTABELECIMENTO,DESC_NATUREZA_JURIDICA,NO_LOGRADOURO,NU_ENDERECO,NO_BAIRRO,CO_CEP,...,UTI_ADULTO_EXIST,UTI_ADULTO_SUS,UTI_PEDIATRICO_EXIST,UTI_PEDIATRICO_SUS,UTI_NEONATAL_EXIST,UTI_NEONATAL_SUS,UTI_QUEIMADO_EXIST,UTI_QUEIMADO_SUS,UTI_CORONARIANA_EXIST,UTI_CORONARIANA_SUS
0,202301,NORDESTE,PE,CABO DE SANTO AGOSTINHO,CASA DE SAUDE SANTA HELENA,HOSPITAL_PRIVADO,AVN PRESIDENTE GETULIO VARGAS,428,CENTRO,54505560,...,0,0,0,0,0,0,0,0,0,0
1,202301,NORDESTE,PE,CABO DE SANTO AGOSTINHO,HOSPITAL MENDO SAMPAIO,HOSPITAL_PÚBLICO,BR 101 SUL KM 33,S/N,CHARNECA,54535430,...,0,0,0,0,0,0,0,0,0,0
2,202301,NORDESTE,PE,CABO DE SANTO AGOSTINHO,MATERNIDADE PADRE GERALDO LEITE BASTOS,HOSPITAL_PÚBLICO,BR 101 SUL KM 23,S/N,PONTE DOS CARVALHOS,54510000,...,0,0,0,0,0,0,0,0,0,0
3,202301,NORDESTE,PE,CABO DE SANTO AGOSTINHO,HOSPITAL SAMARITANO,HOSPITAL_PRIVADO,RUA SEVERINO BEZERRA MARQUES,40,CENTRO,54510460,...,5,0,0,0,0,0,0,0,0,0
4,202301,NORDESTE,PE,CABO DE SANTO AGOSTINHO,HOSPITAL SAO SEBASTIAO,HOSPITAL_PRIVADO,AVN PRESIDENTE GETULIO VARGAS,864,CENTRO,54505560,...,10,0,0,0,0,0,0,0,0,0


In [ ]:
leitos_24 = pd.read_csv("Leitos_2024.csv", encoding="latin-1")
leitos_24 = leitos_24.drop(columns=["MOTIVO_DESABILITACAO","CNES","CO_TIPO_UNIDADE","DS_TIPO_UNIDADE","NATUREZA_JURIDICA","TP_GESTAO","RAZAO_SOCIAL","NO_COMPLEMENTO"])
display(leitos_24.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85225 entries, 0 to 85224
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   COMP                    85225 non-null  int64 
 1   REGIAO                  85225 non-null  object
 2   UF                      85225 non-null  object
 3   MUNICIPIO               85225 non-null  object
 4   NOME_ESTABELECIMENTO    85225 non-null  object
 5   DESC_NATUREZA_JURIDICA  85225 non-null  object
 6   NO_LOGRADOURO           85225 non-null  object
 7   NU_ENDERECO             85225 non-null  object
 8   NO_BAIRRO               85225 non-null  object
 9   CO_CEP                  85225 non-null  int64 
 10  NU_TELEFONE             75403 non-null  object
 11  NO_EMAIL                56878 non-null  object
 12  LEITOS_EXISTENTES       85225 non-null  int64 
 13  LEITOS_SUS              85225 non-null  int64 
 14  UTI_TOTAL_EXIST         85225 non-null  int64 
 15  UT

None

<class 'pandas.core.series.Series'>
RangeIndex: 85225 entries, 0 to 85224
Series name: LEITOS_SUS
Non-Null Count  Dtype
--------------  -----
85225 non-null  int64
dtypes: int64(1)
memory usage: 665.9 KB


0                    CASA DE SAUDE SANTA HELENA
1                        HOSPITAL MENDO SAMPAIO
2        MATERNIDADE PADRE GERALDO LEITE BASTOS
3                           HOSPITAL SAMARITANO
4                        HOSPITAL SAO SEBASTIAO
                          ...                  
85220                    HOSPITAL SAO FRANCISCO
85221                  GRUPO RECANTO UNIDADE II
85222                              HOSPITAL YES
85223               CLINICA MEDICA PERSONNALITE
85224      HOSPITAL E MATERNIDADE NAILA GONCALO
Name: NOME_ESTABELECIMENTO, Length: 85225, dtype: object

None

In [30]:
leitos_25 = pd.read_csv("Leitos_2025.csv", encoding="latin-1" , sep=";")
leitos_25 = leitos_25.drop(columns=["MOTIVO_DESABILITACAO","CNES","CO_TIPO_UNIDADE","DS_TIPO_UNIDADE","NATUREZA_JURIDICA","TP_GESTAO","RAZAO_SOCIAL","NO_COMPLEMENTO"])
display(leitos_25.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50174 entries, 0 to 50173
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   COMP                    50174 non-null  int64 
 1   REGIAO                  50174 non-null  object
 2   UF                      50174 non-null  object
 3   CO_IBGE                 50174 non-null  int64 
 4   MUNICIPIO               50174 non-null  object
 5   NOME_ESTABELECIMENTO    50172 non-null  object
 6   DESC_NATUREZA_JURIDICA  50174 non-null  object
 7   NO_LOGRADOURO           50174 non-null  object
 8   NU_ENDERECO             50174 non-null  object
 9   NO_BAIRRO               50174 non-null  object
 10  CO_CEP                  50174 non-null  int64 
 11  NU_TELEFONE             44350 non-null  object
 12  NO_EMAIL                33857 non-null  object
 13  LEITOS_EXISTENTES       50174 non-null  int64 
 14  LEITOS_SUS              50174 non-null  int64 
 15  UT

None